# 🌳 Decision Predicate Graph (DPG)
This notebook shows a minimal working example of using DPG on **iris** dataset.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from dpg.core import DecisionPredicateGraph
from dpg.visualizer import plot_dpg
from dpg.utils import get_dpg_edge_metrics
from metrics.nodes import NodeMetrics
from metrics.graph import GraphMetrics

#### Training iris

In [ ]:
# Load the iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Fit the model
model = RandomForestClassifier(n_estimators=3, random_state=42)
model.fit(X, y)

#### Extracting DPG

In [ ]:
dpg = DecisionPredicateGraph(
    model=model,
    feature_names=iris.feature_names,
    target_names= iris.target_names
)
dot = dpg.fit(X)
dpg_model, nodes_list = dpg.to_networkx(dot)

# =============================================================================
# METRICS EXTRACTION
# =============================================================================
dpg_metrics = GraphMetrics.extract_graph_metrics(dpg_model, nodes_list,target_names=np.unique(y).astype(str).tolist())
df_nodes = NodeMetrics.extract_node_metrics(dpg_model, nodes_list)


df_edges = get_dpg_edge_metrics(dpg_model, nodes_list)

plot_dpg("dpg_output.png", dot, df_nodes, df_edges, dpg_metrics, save_dir="../datasets", communities=True, class_flag=False)


In [ ]:
import pandas as pd

# Assuming dpg_metrics is already defined and contains "Class Bounds"
class_bounds = dpg_metrics.get("Class Bounds", {})

# Convert to DataFrame: one row per class with constraints joined
df_bounds = pd.DataFrame([
    {"Class": class_name, "Constraints": ", ".join(sorted(bounds))}
    for class_name, bounds in class_bounds.items()
])

# Optional: sort alphabetically by class name
df_bounds = df_bounds.sort_values("Class")
print(df_bounds)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import pandas as pd
import re

# Load Iris dataset
iris = load_iris(as_frame=True)
df_iris = iris.frame
df_iris["target"] = df_iris["target"].apply(lambda x: iris.target_names[x])

# Define signs per class
class_signs = {
    "setosa": "▲",
    "versicolor": "●",
    "virginica": "■"
}

# Extract actual features from the dataset
available_features = df_iris.columns[:-1].tolist()
palette = sns.color_palette("Set2", n_colors=len(df_bounds))

for feature in available_features:
    plt.figure(figsize=(10, 5))
    ax = sns.kdeplot(data=df_iris, x=feature, hue="target", fill=True, common_norm=False, alpha=0.3)
    plt.title(f"KDE + Constraints + Class Signs — {feature}")
    plt.xlabel(feature)
    plt.ylabel("Density")

    feature_min, feature_max = df_iris[feature].min(), df_iris[feature].max()

    for i, row in df_bounds.iterrows():
        class_label = row["Class"].replace("Class ", "")
        constraints = row["Constraints"]
        sign = class_signs.get(class_label, "")
        color = palette[i % len(palette)]

        # Handle bounded ranges: a < feature <= b
        matches_range = re.findall(rf"([\d\.-]+) < {re.escape(feature)} <= ([\d\.-]+)", constraints)
        for lower, upper in matches_range:
            lower, upper = float(lower), float(upper)
            plt.axvline(lower, color=color, linestyle="--", alpha=0.3)
            plt.axvline(upper, color=color, linestyle="--", alpha=0.3)
            plt.fill_betweenx([0, plt.ylim()[1]*0.9+(i*-0.2)], lower, upper, color=color, alpha=0.1)
            plt.text(lower, plt.ylim()[1]*0.95+(i*-0.2), sign, color=color, ha='center')
            plt.text(upper, plt.ylim()[1]*0.95+(i*-0.2), sign, color=color, ha='center')

        # Handle lower-bound only: feature > a
        matches_gt = re.findall(rf"{re.escape(feature)} > ([\d\.-]+)", constraints)
        for lower in matches_gt:
            lower = float(lower)
            plt.axvline(lower, color=color, linestyle="--", alpha=0.3)
            plt.fill_betweenx([0, plt.ylim()[1]*0.9+(i*-0.2)], lower, feature_max, color=color, alpha=0.1)
            plt.text(lower, plt.ylim()[1]*0.95+(i*-0.2), sign, color=color, ha='center')

        # Handle upper-bound only: feature <= b
        matches_le = re.findall(rf"{re.escape(feature)} <= ([\d\.-]+)", constraints)
        for upper in matches_le:
            upper = float(upper)
            plt.axvline(upper, color=color, linestyle="--", alpha=0.3)
            plt.fill_betweenx([0, plt.ylim()[1]*0.9+(i*-0.2)], feature_min, upper, color=color, alpha=0.1)
            plt.text(upper, plt.ylim()[1]*0.95+(i*-0.2), sign, color=color, ha='center')

    plt.tight_layout()
    plt.show()
